In [39]:
# IMPORT Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
from tensorflow.keras.utils import to_categorical
import numpy as np

In [40]:
# Import the Australian Road Death Dataset
ardd_data = pd.read_csv('../Resources/ardd_fatal_crashes.csv')
ardd_data

/var/folders/h4/v2g57xvj6mz75vxd96qdxsqw0000gn/T/ipykernel_47825/2073364016.py:2: DtypeWarning: Columns (11,12,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  ardd_data = pd.read_csv('../Resources/ardd_fatal_crashes.csv')


,Crash ID,State,Month,Year,Dayweek,Time,Crash Type,Number Fatalities,Bus \nInvolvement,Heavy Rigid Truck Involvement,Articulated Truck Involvement,Speed Limit,National Remoteness Areas,SA4 Name 2016,National LGA Name 2017,National Road Type,Christmas Period,Easter Period,Day of week,Time of Day
0,20221049,NSW,3,2022,Tuesday,06:50,Single,1,No,Yes,No,90,Inner Regional Australia,Capital Region,Eurobodalla,Arterial Road,No,No,Weekday,Day
1,20222018,Vic,3,2022,Friday,13:05,Multiple,1,-9,-9,-9,-9,Major Cities of Australia,Melbourne - Inner,Port Phillip (C),Local Road,No,No,Weekday,Day
2,20224006,SA,3,2022,Friday,09:30,Multiple,1,No,No,No,80,Outer Regional Australia,South Australia - Outback,Port Augusta (C),National or State Highway,No,No,Weekday,Day
3,20225008,WA,3,2022,Saturday,06:00,Single,1,No,No,No,110,NaN,NaN,NaN,NaN,No,No,Weekend,Day
4,20226010,Tas,3,2022,Tuesday,11:47,Single,1,No,No,No,100,Outer Regional Australia,Launceston and North East,Break O'Day (M),National or State Highway,No,No,Weekday,Day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48116,19892204,Vic,1,1989,Friday,13:20,Single,1,No,No,No,60,NaN,NaN,NaN,NaN,No,No,Weekday,Day
48117,19893326,Qld,1,1989,Monday,09:00,Multiple,1,No,-9,No,60,NaN,NaN,NaN,NaN,No,No,Weekday,Day
48118,19892576,Vic,1,1989,Friday,17:15,Multiple,1,No,No,No,60,NaN,NaN,NaN,NaN,No,No,Weekday,Day
48119,19891122,NSW,1,1989,Tuesday,14:10,Multiple,1,No,-9,No,60,NaN,NaN,NaN,NaN,No,No,Weekday,Day


In [41]:
#check the shape of the original CSV
ardd_data.shape

(48121, 20)

In [42]:
ardd_data = ardd_data.filter(['State','Crash Type','Bus \nInvolvement','Heavy Rigid Truck Involvement','Articulated Truck Involvement','Speed Limit','Christmas Period','Easter Period','Day of week'])
ardd_data.head(20)

,State,Crash Type,Bus \nInvolvement,Heavy Rigid Truck Involvement,Articulated Truck Involvement,Speed Limit,Christmas Period,Easter Period,Day of week
0,NSW,Single,No,Yes,No,90,No,No,Weekday
1,Vic,Multiple,-9,-9,-9,-9,No,No,Weekday
2,SA,Multiple,No,No,No,80,No,No,Weekday
3,WA,Single,No,No,No,110,No,No,Weekend
4,Tas,Single,No,No,No,100,No,No,Weekday
5,NSW,Multiple,No,No,No,70,No,No,Weekend
6,NSW,Single,No,No,No,60,No,No,Weekend
7,Vic,Multiple,-9,-9,-9,60,No,No,Weekend
8,Vic,Multiple,-9,-9,-9,100,No,No,Weekend
9,Qld,Single,No,No,No,100,No,No,Weekday


In [43]:
# according to the documentation, -9 would mean "Unknown", however after data exploration those data will be of no use for our ML model
ardd_data = ardd_data.replace("-9", np.nan)

In [44]:
#drop rows containing null values
ardd_data = ardd_data.dropna()

In [45]:
#check shape 
ardd_data.shape

(29464, 9)

In [46]:
ardd_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29464 entries, 0 to 48118
Data columns (total 9 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   State                          29464 non-null  object
 1   Crash Type                     29464 non-null  object
 2   Bus 
Involvement               29464 non-null  object
 3   Heavy Rigid Truck Involvement  29464 non-null  object
 4   Articulated Truck Involvement  29464 non-null  object
 5   Speed Limit                    29464 non-null  object
 6   Christmas Period               29464 non-null  object
 7   Easter Period                  29464 non-null  object
 8   Day of week                    29464 non-null  object
dtypes: object(9)
memory usage: 2.2+ MB


In [47]:
ardd_data = ardd_data.rename(columns={'Crash Type': 'crash_type','Bus \nInvolvement': 'bus_involvement','Heavy Rigid Truck Involvement': 'heavy_rigid_truck_involvement','Articulated Truck Involvement': 'articulated_truck_involvement','Speed Limit': 'speed_limit','Christmas Period': 'christmas_period','Easter Period': 'easter_period','Day of week': 'day_of_week'})
ardd_data

,State,crash_type,bus_involvement,heavy_rigid_truck_involvement,articulated_truck_involvement,speed_limit,christmas_period,easter_period,day_of_week
0,NSW,Single,No,Yes,No,90,No,No,Weekday
2,SA,Multiple,No,No,No,80,No,No,Weekday
3,WA,Single,No,No,No,110,No,No,Weekend
4,Tas,Single,No,No,No,100,No,No,Weekday
5,NSW,Multiple,No,No,No,70,No,No,Weekend
...,...,...,...,...,...,...,...,...,...
48108,Vic,Single,No,No,No,60,No,No,Weekday
48109,Vic,Single,No,No,No,60,No,No,Weekday
48114,Vic,Single,No,No,No,60,No,No,Weekday
48116,Vic,Single,No,No,No,60,No,No,Weekday


In [48]:
cleaned_ardd_data2 = cleaned_ardd_data2.rename(columns={'Crash Type': 'crash_type','Speed Limit': 'speed_limit','National Remoteness Areas': 'national_remoteness_area','National Road Type': 'national_road_type','Christmas Period': 'christmas_period','Easter Period': 'easter_period','Day of week': 'day_of_week'})
cleaned_ardd_data2

NameError: name 'cleaned_ardd_data2' is not defined

In [49]:
# Set features. 
X = ardd_data.drop("crash_type", axis=1)
y = ardd_data["crash_type"]
print(X.shape, y.shape)

(29464, 8) (29464,)


In [50]:
# Set features. 
X2 = cleaned_ardd_data2.drop("crash_type", axis=1)
y2 = cleaned_ardd_data2["crash_type"]
print(X2.shape, y2.shape)

NameError: name 'cleaned_ardd_data2' is not defined

In [51]:
X

,State,bus_involvement,heavy_rigid_truck_involvement,articulated_truck_involvement,speed_limit,christmas_period,easter_period,day_of_week
0,NSW,No,Yes,No,90,No,No,Weekday
2,SA,No,No,No,80,No,No,Weekday
3,WA,No,No,No,110,No,No,Weekend
4,Tas,No,No,No,100,No,No,Weekday
5,NSW,No,No,No,70,No,No,Weekend
...,...,...,...,...,...,...,...,...
48108,Vic,No,No,No,60,No,No,Weekday
48109,Vic,No,No,No,60,No,No,Weekday
48114,Vic,No,No,No,60,No,No,Weekday
48116,Vic,No,No,No,60,No,No,Weekday


In [52]:
y.unique()

array(['Single', 'Multiple'], dtype=object)

In [53]:
from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder()

transformed_X = onehotencoder.fit_transform(X[["State", "bus_involvement","heavy_rigid_truck_involvement", "articulated_truck_involvement", "speed_limit","christmas_period", "easter_period","day_of_week"]])
print(transformed_X)

TypeError: Encoders require their input to be uniformly strings or numbers. Got ['int', 'str']

In [54]:
transformed_X2 = onehotencoder.fit_transform(X2[["State", "Month","Year", "speed_limit", "national_remoteness_area", "national_road_type","christmas_period", "easter_period","day_of_week"]])
print(transformed_X2)

NameError: name 'X2' is not defined

In [55]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(y)
encoded_y = label_encoder.transform(y)

In [ ]:
label_encoder.fit(y2)
encoded_y2 = label_encoder.transform(y2)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    transformed_X, encoded_y, random_state=1)


In [ ]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(
    transformed_X2, encoded_y2, random_state=1)


In [ ]:
X_train

In [ ]:
y_train

In [ ]:
y_test

In [ ]:
X_test

In [ ]:
from sklearn import tree

In [ ]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_train, y_train)

In [ ]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train2, y_train2)
clf.score(X_train2, y_train2)

In [ ]:
clf = clf.fit(X_train, y_train)
clf.score(X_train, y_train)

In [ ]:
print(f"Training Data Score: {clf.score(X_train, y_train)}")
print(f"Testing Data Score: {clf.score(X_test, y_test)}")

In [ ]:
print(f"Training Data Score: {clf.score(X_train2, y_train2)}")
print(f"Testing Data Score: {clf.score(X_test2, y_test2)}")

In [ ]:
y.value_counts()

In [ ]:
clf.feature_importances_

In [ ]:
top_df = pd.DataFrame({"column":X_train.columns, "importance":clf.feature_importances_}).sort_values(by="importance")